# GRU

*Gated Recurent Unit*

ゲート付き回帰型ユニット

RNN層にゲートと呼ばれる機構を追加して長期的な文脈が保持できるようになったもの。  
RNN同様、「GRU」が層を表している場合とそれが組み込まれたNNを表している場合がある。前章ではそれらの表記を分けたが、本章では分けないので、察して読んで。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
from typing import List

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
from dlprog import train_progress


---

## ゲート

あるデータをどれくらい通すかを示したもの。具体的には対称のデータと同じサイズのベクトル。0-1の値をとる。  
NNで実装してみる。

In [2]:
class Gate(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

入力されたデータを線形変化し、sigmoid関数に入力するだけ。このモデルにあるデータを入力したときの出力値が、そのデータのゲートとなる。  
ゲートを元のデータに掛けることで、元のデータの一部を**通した**ということになる。

In [3]:
input_size = 3
gate = Gate(input_size)

x = torch.randn(input_size)
y = x * gate(x)
print('input:', x)
print('gate:', gate(x))
print('output:', y)

input: tensor([-0.5275, -0.4275, -0.1577])
gate: tensor([0.4774, 0.4499, 0.4586], grad_fn=<SigmoidBackward0>)
output: tensor([-0.2518, -0.1923, -0.0723], grad_fn=<MulBackward0>)



---

## GRUの構造

GRUの構造を見ていこう。

一旦RNNの復習をしよう。


RNNはある時間$t$の入力$x_t$に対して以下のような演算で出力値$h_t$を決定する。

$$
h_t = \mathrm{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

この$x_t$と$h_{t-1}$の全結合の部分は$\mathrm{fc}(x,h)$で表すことにしよう。

$$
\begin{align}
h_t &= \mathrm{tanh}(\mathrm{fc}(x_t,h_{t-1})) \\
\mathrm{fc}(x,h) &= W_x x + b_x + W_h h + b_h
\end{align}
$$

んで、$\mathrm{fc}(x,h)$の実装もしておこう。

In [4]:
class FullyConnected(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc_input = nn.Linear(input_size, hidden_size)
        self.fc_hidden = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h):
        return self.fc_input(x) + self.fc_hidden(h)

では、GRUの構造を見ていこう。  
GRUは以下のような演算で出力値$h_t$を決定する。

$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,h_{t-1})) \\
z_t &= \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
\end{align}
$$

$\sigma(x)$はsigmoid関数。

RNNでは新たなデータ$\tilde h_t$がそのまま出力されていた。  
GRUでは、新たなデータ$\tilde h_t$を古いデータ$h_{t-1}$に足して出力する。そして、その際の比率をゲート$z_t$で決める。この$z_t$は$h_{t-1}$をどれだけ通すかを表す。

In [5]:
class SimpleGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc = FullyConnected(input_size, hidden_size)
        self.gate = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        h_new = F.tanh(self.fc(x, h))
        z = self.gate(x)
        h = (1 - z) * h_new + z * h
        return h

このように、GRUではゲートを用いて新たなデータをどれだけ取り入れるべきか、そして古いデータをどれだけ捨てるか考えることが出来る。  
この枠組みの下で学習を行うことで、長期的に保持すべきデータをしっかりと保持できるようになることが期待される。

ちなみに、上記のモデルは一般的なGRUを私が簡略化したもの。  
一般的なGRUは、上記のモデルにゲートを一つ追加した以下のモデルである。


$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,r_t \odot h_{t-1})) \\
z_t &= \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
r_t &= \sigma(\mathrm{fc}_{r}(x_t,h_{t-1})) \\
\end{align}
$$

新なデータ$\tilde h_t$を生成する際に、古いデータ$h_{t-1}$をどれだけ考慮するかを決めるゲート$r_t$が追加されている。

In [6]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc_input = FullyConnected(input_size, hidden_size)
        self.gate_update = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.gate_reset = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        r = self.gate_reset(x, h)
        h_new = F.tanh(self.fc_input(x, r * h))
        z = self.gate_update(x)
        h = (1 - z) * h_new + z * h
        return h

また、RNN同様、PyTorchにクラスとして`torch.nn.GRU`が用意されている:  
[GRU — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html)

In [7]:
gru = nn.GRU(input_size, input_size)


---

## LSTM

*Long Short-Term Memory*

長短期記憶

GRUの進化版。考え方はGRUと同じで、RNNにゲートを取り入れてイイ感じにしたもの。  
ちなみに、GRUよりLSTMの方が先に提案されている。GRUはLSTMの簡易版として後から提案された。

LSTMには出力する隠れ状態$h_t$だけでなく、**記憶セル**と呼ばれる変数$c_t$を持つ。記憶セルはLSTMの外に出力されることはなく、LSTM内部でのみ使用される。

まず簡単に文字で説明する。  
記憶セル$c_t$がGRUでの隠れ状態$h_t$に当たり、ゲートを用いた不要な情報の削除と新たな情報の追加が行われる。なおゲートの生成には入力$x_t$と前の隠れ状態$h_{t-1}$を用いる（記憶セルは用いない）。そしてこの記憶セルを活性化関数に通したものをLSTMの出力=隠れ状態$h_t$とする。

具体的な構造を見てみよう。

$$
\begin{align}
h_t &= o_t \odot \mathrm{tanh}(c_t) \\
c_t &= f_t \odot c_{t-1} + i_t \odot \tilde c_t \\
\tilde c_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde c}(x_t,h_{t-1})) \\
i_t &= \sigma(\mathrm{fc}_{i}(x_t,h_{t-1})) \\
f_t &= \sigma(\mathrm{fc}_{f}(x_t,h_{t-1})) \\
o_t &= \sigma(\mathrm{fc}_{o}(x_t,h_{t-1})) \\
\end{align}
$$


- $\tilde c_t$: 新たな情報。
- $i_t$: inputゲート。新たな情報$\tilde c_t$をどれだけ取り入れるかを決める。
- $f_t$: forgetゲート。古い情報$c_{h-1}$をどれだけ保持するかを決めるゲート。
- $o_t$: outputゲート。出力する隠れ状態の量を決めるゲート。

GRUでは1つのゲートを用いて新たな情報と古い情報の比率を決めていたが、LSTMでは別々のゲートを用いて決める。

実装は以下の通り。

In [8]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.gate_input = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.gate_forget = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.gate_output = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.fc = FullyConnected(input_size, hidden_size)

    def forward(self, x, h, c):
        c_new = F.tanh(self.fc(x, h))
        i = self.gate_input(x, h)
        f = self.gate_forget(x, h)
        o = self.gate_output(x, h)
        c = f * c + i * c_new
        h = o * F.tanh(c)
        return h, c

PyTorchにも`torch.nn.LSTM`が用意されている:  
[LSTM — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)


---

## LSTMを用いた言語モデル

せっかくなので、LSTMで言語モデルを作ってみよう。RNNLMのRNN層をLSTMに変更するだけ。

In [9]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
textfile = 'data/jawiki.txt'
data = open(textfile).readlines()
data = data[:10000]

model_prefix = 'models/jawiki_tokenizer'
sp = spm.SentencePieceProcessor(f'{model_prefix}.model')
data_ids = sp.encode(data)
n_vocab = len(sp)

bos_id = sp.bos_id()
eos_id = sp.eos_id()
for ids in data_ids:
    if ids:
        ids[0] = bos_id
        ids[-1] = eos_id

class TextLoader:
    def __init__(self, data_ids):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __iter__(self):
        random.shuffle(self.data)
        for text in self.data:
            yield text[:-1], text[1:]

    def __len__(self):
        return self.n_data

dataloader = TextLoader(data_ids)
sample_y, sample_t = next(iter(dataloader))
sample_y[:10], sample_t[:10] # example

(tensor([   1, 2966,    7, 7435, 1705,  499,   17,   68,   78,   92]),
 tensor([2966,    7, 7435, 1705,  499,   17,   68,   78,   92,  378]))

In [21]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, hc=None):
        x = self.embedding(x) # (seq_len, embed_size)
        y, (h, c) = self.lstm(x, hc) # (seq_len, hidden_size)
        y = self.fc(y) # (seq_len, n_vocab)
        return y, (h, c)

In [22]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            x, t = x.to(device), t.to(device)
            optimizer.zero_grad()
            y, _ = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [23]:
hidden_size = 1024
model = LanguageModel(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train(model, optimizer, 100, 10)